In [ ]:
GG_MC_URL = "https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actress_in_a_Motion_Picture_%E2%80%93_Musical_or_Comedy"

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(requests.get(GG_MC_URL).text, 'html.parser')


In [4]:
tables = soup.find_all('table', class_='wikitable sortable')

In [5]:
def isWinner(row):
    return bool(row.find('td',{'style': 'background:#B0C4DE;'})) 

In [6]:
df = pd.DataFrame(columns=['ano','nome','personagem','filme','ganhou'])


for table in tables:
    rows = table.find_all('tr')
    current_year = None  
    current_filme = None
    
    for row in rows[1:]:
        ano_th = row.find('th', {'style': 'text-align:center;'}) 
        if not ano_th:
             ano_th = row.find('th')
        
        # Se encontrou um th com o estilo, atualiza o ano atual
        if ano_th is not None and ano_th.find('a'):
            current_year = ano_th.find('a').text 
        
        ano_atual = current_year
        #print(f"Ano para esta linha: {ano_atual}")
        
        tds = row.find_all('td')
        nome = tds[0].find('a').text if tds[0].find('a') else None
        if nome is None:
            nome = tds[0].text.strip()
        personagem = tds[1].find('a').text if tds[1].find('a') else None
        if personagem is None:
             personagem = tds[1].text.strip()
        if len(tds) > 2:
                filme = tds[2].find('a').text if tds[2].find('a') else None
                # Atualiza o filme atual para uso em linhas futuras
                if filme:
                    current_filme = filme
                else:
                    # Usa o filme armazenado anteriormente
                    filme = current_filme
    
        ganhou = isWinner(row)
    
        df = pd.concat(
            [
                df,
                pd.DataFrame(
                    {
                        'ano': [ano_atual],
                        'nome': [nome],
                        'personagem': [personagem],
                        'filme': [filme],
                        'ganhou': [ganhou]
                    }
                )
            ]
    )

In [7]:
df

,ano,nome,personagem,filme,ganhou
0,1950,Judy Holliday,"Emma ""Billie"" Dawn",Born Yesterday,True
0,1950,Spring Byington,Louisa Norton,Louisa,False
0,1950,Betty Hutton,Annie Oakley,Annie Get Your Gun,False
0,1951,June Allyson,Cynthia Potter,Too Young to Kiss,True
0,1952,Susan Hayward,Jane Froman,With a Song in My Heart,True
...,...,...,...,...,...
0,2024,Amy Adams,Mother,Nightbitch,False
0,2024,Cynthia Erivo,Elphaba Thropp,Wicked,False
0,2024,Karla Sofía Gascón,"Emilia Pérez / Juan ""Manitas"" Del Monte",Emilia Pérez,False
0,2024,Mikey Madison,"Anora ""Ani"" Mikheeva",Anora,False


In [ ]:
df.to_csv('data/original/gg_mc.csv', index=False)